# 解析PDF

为了保证数据干净，需要把处理好的txt中前9行和 倒数前四行删掉

In [4]:
!pip install PyMuPDF PyPDF2 tiktoken

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ---------------------------------------- 0.0/884.8 kB ? eta -:--:--
     ----------- ---------------------------- 262.1/884.8 kB ? eta -:--:--
     --------------------------------- ---- 786.4/884.8 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 884.8/884.8 kB 2.1 MB/s eta 0:00:00


In [1]:
from Chunk.datachunk import TextChunker
from Embedding.model import ChineseBERTEmbedding
from VectorDatabase.database import Vectordatabase

e:\miniconda\envs\RAG\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
e:\miniconda\envs\RAG\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [11]:
import fitz  # PyMuPDF

def pdf_to_txt(pdf_path, txt_path):
    try:
        # 打开 PDF 文件
        pdf_document = fitz.open(pdf_path)
        text_content = ""

        # 遍历每一页提取文本 : 从第12页开始，后面4页是广告
        for page_num in range(11,len(pdf_document) - 4):
            page = pdf_document[page_num]
            text_content += page.get_text() + "\n"  # 获取当前页的文字内容

        # 将提取的文本保存到 TXT 文件
        with open(txt_path, "w", encoding="utf-8") as txt_file:
            txt_file.write(text_content)

        print(f"成功将 PDF 内容保存为 TXT 文件：{txt_path}")
    except Exception as e:
        print(f"发生错误：{e}")

# 文件路径
pdf_file = "Book/齐民要术-石声汉译注.pdf"  # 输入 PDF 文件路径
txt_file = "Book/齐民要术-石声汉译注.txt"  # 输出 TXT 文件路径

# 执行转换
pdf_to_txt(pdf_file, txt_file)

成功将 PDF 内容保存为 TXT 文件：Book/齐民要术-石声汉译注.txt


In [12]:
def clean_and_combine_txt(input_path, output_path):
    try:
        with open(input_path, "r", encoding="utf-8") as file:
            lines = file.readlines()

        # 删除空行并去掉每行的首尾空格
        cleaned_lines = [line.strip() for line in lines if line.strip()]

        # 拼接所有行，按段落合并（用空格连接）
        combined_text = " ".join(cleaned_lines)

        # 将处理后的文本写入新文件
        with open(output_path, "w", encoding="utf-8") as file:
            file.write(combined_text)

        print(f"成功处理文本并保存至：{output_path}")
    except Exception as e:
        print(f"发生错误：{e}")

# 文件路径
input_txt = "Book/齐民要术-石声汉译注.txt"  # 输入 TXT 文件路径
output_txt = "Book/齐民要术.txt"  # 输出处理后 TXT 文件路径

# 执行清理和拼接
clean_and_combine_txt(input_txt, output_txt)

成功处理文本并保存至：Book/齐民要术.txt


In [4]:
def analyze_text(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()

        # 统计总字数（去掉空格后的字符总数）
        total_characters = len(content.replace(" ", "").replace("\n", ""))

        # 统计总行数
        lines = content.splitlines()
        total_lines = len(lines)

        # 统计段落数（按空行分割段落）
        paragraphs = [para.strip() for para in content.split("\n\n") if para.strip()]
        total_paragraphs = len(paragraphs)

        # 统计单词数（按空格分割的词汇总数）
        words = content.split()
        total_words = len(words)

        # 打印统计结果
        print(f"总字数（去空格）：{total_characters}")
        print(f"总行数：{total_lines}")
        print(f"总段落数：{total_paragraphs}")
        print(f"总单词数：{total_words}")

        # 返回统计结果（如果需要进一步使用）
        return {
            "total_characters": total_characters,
            "total_lines": total_lines,
            "total_paragraphs": total_paragraphs,
            "total_words": total_words,
        }
    except Exception as e:
        print(f"发生错误：{e}")

# 文件路径
txt_file = "Book/齐民要术.txt"  # 输入 TXT 文件路径

# 执行统计分析
stats = analyze_text(txt_file)

总字数（去空格）：696830
总行数：1
总段落数：1
总单词数：49507


In [7]:
with open(txt_file,"r", encoding="utf-8") as file:
    content = file.read()
    chunker = TextChunker(content)
    chunks = chunker.chunk_text(chunk_size=200, overlap=150)
chunks

list

# 加载模型，把文本向量化存储

In [8]:
#嵌入模型：将文本向量化
embedding_model = ChineseBERTEmbedding()
#向量库
database = Vectordatabase(chunks)

#对字块列表进行，批量的embedded编码，传入embedding模型，返回一个向量列表
Vectors = database.get_vector(embedding_model)
#把向量列表存储到json文件中，把子块列表也存储到json文件
database.persist()

100%|██████████| 18016/18016 [08:50<00:00, 33.94it/s]


# 测试

In [2]:
text = "如何选用优质的稻种"
#嵌入模型：将文本向量化
embedding_model = ChineseBERTEmbedding()

database = Vectordatabase()

database.load_vector()

result = database.query(text,embedding_model,5)
result

['稻田，水稻的田间管理，水稻的收获 及稻种的储藏，舂稻米的注意事项等都作了详细 地记述。 11.3.1中水稻先浸种到“芽长二分”再播种的记 叙，可能是有关“浸种”的最早文字记载。 水稻，产量高。因为它能靠水层保持比较稳 定的温度，所以对水的要求很严格。水过多过少 都不能生长，含泥过多的浊水也对生长不利。水 稻种植的关键之一是管好水。本篇详细介绍了',
 '，也可以晒作干菜。现代有学者认 为：蜀芥，可能是大芥菜；芥子，可能是小芥 菜。芸薹则是油菜的一种，植株较矮小，现在北 方地区还普遍栽种。标题注中所说的水苏、劳蒩 实际上与“芥”无涉。 贾思勰介绍了它们的播种（季节、对土质的 要求、种子用量等），田间管理。特别提醒：用 叶子的在七月半下种，十月收；预备收子用的， 则在二三月种，五',
 '成熟有早有晚，谷苗茎秆有高 有矮，收下的种实有多有少，植株质地性状有的 坚强有的软弱，谷米的味道有的好有的不好，谷 粒舂成米时有的耗折多，有的耗折少。成熟早 的，茎秆矮，但收获量大；成熟晚的，茎秆高而收获 量少。植株坚强的，都矮小，黄谷就是这样；植株软 弱的，就长得高，青谷、白谷、黑谷',
 '�坚强的，都矮小，黄谷就是这样；植株软 弱的，就长得高，青谷、白谷、黑谷是这样。收获量 少的味道好，但折耗多；收获量高的，味道不好而折 耗少。另一方面，种谷的地，肥力有高低，好地宜于 晚些种，瘦地必须早种。好地不仅宜于晚些种，种早 了也不会有妨害；瘦地必须早种，种晚了一定没有种 实可收。不同的地形条件，也应当作不同的',
 '谷子一样。 5.3.3\u3000收刈欲晚。性不零落，早刈损实。 【译文】 5.3.3\u3000收割要迟。粱和秫都不落粒，收割太 早，便长不饱满，种实有损失。 大豆第六 【题解】 6.1.1—6.1.5为篇标题注。 大豆是我国原产。考古学的成果证明，大豆 在我国的栽培已有悠久的历史。大豆蛋白质最接 近动物性蛋白质的标准，营养价值']

In [17]:
import time ,jwt,requests
# LLM智谱大模型API
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)
    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

def ask_glm(key, model, max_tokens, temperature, content):
    # 智谱大模型
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': generate_token(key, 1000)
    }

    data = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": content
    }

    response = requests.post(url, headers=headers, json=data)

    return response.json()
#你的API
text = text + str(result)
key = "8b00afab1e7440a29b2cc9dba6e17f94.pjG00RsmnmyiHqsG"
content = [{"role": "user", "content": text}]
full_response = ask_glm(key, "glm-3-turbo", 512, 0.8,content)

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '您好，看起来您可能是在询问如何选择优质的稻种。在《齐民要术》中，贾思勰详细讨论了稻种的挑选和种植方法，以下是一些关键点：\n\n1. 成熟早晚：成熟早的稻种茎秆较矮，但收获量大；成熟晚的稻种茎秆高，但收获量少。\n\n2. 植株强弱：植株坚强的稻种都矮小，如黄谷；植株软弱的稻种就长得高，如青谷、白谷、黑谷。\n\n3. 味道与折耗：收获量少的味道好，但折耗多；收获量高的味道不好，但折耗少。\n\n4. 土壤肥力：肥力高的土壤宜于晚些种植，瘦地必须早种。\n\n5. 地形条件：不同的地形条件也应当作不同的安排。\n\n6. 浸种方法：11.3.1中提到水稻先浸种到“芽长二分”再播种，可能是最早的浸种技术记载。\n\n7. 田间管理：详细介绍了水稻的田间管理，包括水分管理、病虫害防治等。\n\n8. 收获与储藏：讲述了水稻的收割与储藏方法。\n\n选择稻种时，要根据当地的土壤、气候条件和种植习惯，选择合适的品种，并在种植过程中注意田间管理，以获得理想的产量和品质。希望这些信息对您有所帮助。如果您还有其他问题，欢迎继续咨询。',
    'role': 'assistant'}}],
 'created': 1736845545,
 'id': '202501141705337e14a11acfbd4f14',
 'model': 'glm-3-turbo',
 'request_id': '202501141705337e14a11acfbd4f14',
 'usage': {'completion_tokens': 326,
  'prompt_tokens': 8019,
  'total_tokens': 8345}}

In [18]:
full_response['choices'][0]['message']['content']

'您好，看起来您可能是在询问如何选择优质的稻种。在《齐民要术》中，贾思勰详细讨论了稻种的挑选和种植方法，以下是一些关键点：\n\n1. 成熟早晚：成熟早的稻种茎秆较矮，但收获量大；成熟晚的稻种茎秆高，但收获量少。\n\n2. 植株强弱：植株坚强的稻种都矮小，如黄谷；植株软弱的稻种就长得高，如青谷、白谷、黑谷。\n\n3. 味道与折耗：收获量少的味道好，但折耗多；收获量高的味道不好，但折耗少。\n\n4. 土壤肥力：肥力高的土壤宜于晚些种植，瘦地必须早种。\n\n5. 地形条件：不同的地形条件也应当作不同的安排。\n\n6. 浸种方法：11.3.1中提到水稻先浸种到“芽长二分”再播种，可能是最早的浸种技术记载。\n\n7. 田间管理：详细介绍了水稻的田间管理，包括水分管理、病虫害防治等。\n\n8. 收获与储藏：讲述了水稻的收割与储藏方法。\n\n选择稻种时，要根据当地的土壤、气候条件和种植习惯，选择合适的品种，并在种植过程中注意田间管理，以获得理想的产量和品质。希望这些信息对您有所帮助。如果您还有其他问题，欢迎继续咨询。'